In [13]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

## Split the Data into Training and Testing Sets

In [14]:
# Read the CSV file 
nba_df = pd.read_csv("C:/Users/tsswi/OneDrive/Desktop/DU_Classwork/Module_23_Project4/merged_df.csv")

In [15]:
# Review the DataFrame
nba_df.head(10)

,GAME_DATE_EST,game_id,home_team,away_team,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,over_odds,PTS_comb_actual,book_name_under,book_id_under,total1_under,under_odds,point_average_last10,point_againts_average_last10,away_point_average_last10,away_point_againts_average_last10
0,2017-09-30,11700001,1610612744,1610612743,2017,102.0,0.411,0.833,0.121,17.0,...,-110.0,210.0,Pinnacle Sports,238,222.5,108.0,116.226415,104.320755,111.176893,110.534142
1,2017-09-30,11700002,1610612747,1610612750,2017,99.0,0.441,0.706,0.167,27.0,...,-110.0,207.0,Bookmaker,93,216.0,-102.0,104.435540,111.298407,105.205303,105.693687
2,2017-11-01,21700111,1610612752,1610612745,2017,97.0,0.433,0.846,0.308,23.0,...,-107.0,216.0,Pinnacle Sports,238,215.5,-102.0,99.916667,107.183333,110.787037,104.050926
3,2017-11-01,21700107,1610612755,1610612737,2017,119.0,0.465,0.560,0.371,36.0,...,-104.0,228.0,5Dimes,19,209.0,-105.0,105.028571,107.771429,101.091912,104.283088
4,2017-11-01,21700108,1610612764,1610612756,2017,116.0,0.446,0.800,0.333,20.0,...,-105.0,238.0,Heritage,169,228.0,-105.0,109.972222,104.277778,105.366667,115.866667
5,2017-11-01,21700110,1610612748,1610612741,2017,97.0,0.410,0.895,0.387,21.0,...,-105.0,188.0,Pinnacle Sports,238,196.5,-105.0,102.906250,104.125000,96.233333,106.366667
6,2017-11-01,21700115,1610612762,1610612757,2017,112.0,0.378,0.861,0.448,16.0,...,-105.0,215.0,5Dimes,19,192.0,-105.0,100.800000,95.400000,109.583333,99.666667
7,2017-11-01,21700109,1610612738,1610612758,2017,113.0,0.488,0.842,0.415,30.0,...,-101.0,199.0,Bovada,999996,196.0,-110.0,103.416667,94.816667,93.112500,102.887500
8,2017-11-01,21700113,1610612740,1610612750,2017,98.0,0.429,0.857,0.258,27.0,...,-105.0,202.0,Pinnacle Sports,238,225.0,-105.0,105.172619,108.437500,108.260417,106.833333
9,2017-11-01,21700114,1610612743,1610612761,2017,129.0,0.534,0.655,0.500,35.0,...,-105.0,240.0,Pinnacle Sports,238,217.5,-105.0,110.292271,108.974638,106.501374,101.627747


In [16]:
# Rename columns 
nba_df.rename(columns = {'point_againts_average_last10':'point_against_average_last10', 'away_point_againts_average_last10':'away_point_against_average_last10'}, inplace = True)

In [6]:
# Drop non-beneficial columns if applicable 
# nba_df.drop(columns=[''], inplace=True)

In [19]:
# Determine the number of unique values in each column.
nba_df.nunique()

GAME_DATE_EST                         198
game_id                              1210
home_team                              30
away_team                              30
SEASON                                  1
PTS_home                               68
FG_PCT_home                           223
FT_PCT_home                           174
FG3_PCT_home                          219
AST_home                               29
REB_home                               37
PTS_away                               68
FG_PCT_away                           225
FT_PCT_away                           181
FG3_PCT_away                          207
AST_away                               33
REB_away                               42
HOME_TEAM_WINS                          2
book_name_over                          9
book_id_over                            9
total1_over                            92
over_odds                              21
PTS_comb_actual                       106
book_name_under                   

In [20]:
# Look at actual game points value counts for binning
nba_df['PTS_comb_actual'].value_counts()

215.0    34
207.0    33
205.0    27
213.0    27
211.0    27
         ..
255.0     1
250.0     1
158.0     1
171.0     1
166.0     1
Name: PTS_comb_actual, Length: 106, dtype: int64

In [24]:
# Choose a cutoff value and create a list of actual game points to be replaced
# use the variable name `pts_types_to_replace`
pts_types = pd.DataFrame(nba_df['PTS_comb_actual'].value_counts()).reset_index()

pts_types_to_replace = []

for index, row in pts_types.iterrows():
    if row['PTS_comb_actual'] < 10:
        pts_types_to_replace.append(row['index'])
pts_types_to_replace 


# Replace in dataframe
for pts_types in pts_types_to_replace:
    nba_df['PTS_comb_actual'] = nba_df['PTS_comb_actual'].replace(pts_types,"Other")

# Check to make sure binning was successful
nba_df['PTS_comb_actual'].value_counts()

Other    189
215.0     34
207.0     33
213.0     27
206.0     27
205.0     27
211.0     27
217.0     26
192.0     25
208.0     25
209.0     25
223.0     25
204.0     24
202.0     24
218.0     24
221.0     24
224.0     23
201.0     23
226.0     23
227.0     22
225.0     21
193.0     21
210.0     21
233.0     20
187.0     20
220.0     19
212.0     19
203.0     19
200.0     19
214.0     18
195.0     17
197.0     17
231.0     17
191.0     16
190.0     16
196.0     16
232.0     15
219.0     15
234.0     15
198.0     14
189.0     14
228.0     14
216.0     14
237.0     14
241.0     13
194.0     13
240.0     13
229.0     13
199.0     13
188.0     13
230.0     11
236.0     11
235.0     11
242.0     11
Name: PTS_comb_actual, dtype: int64

In [ ]:
# Look at YYYY value counts for binning
yyyy_counts = nba_df['YYYY'].value_counts()
yyyy_counts

In [ ]:
# Choose a cutoff value and create a list of YYYY to be replaced
# use the variable name `yyyy_to_replace`
yyyy_types = pd.DataFrame(nba_df['YYYY'].value_counts()).reset_index()

yyyy_to_replace = []

for index, row in yyyy.iterrows():
    if row['YYYY'] < #y:
        yyyy_to_replace.append(row['index'])
yyyy_to_replace 

# Replace in dataframe
for yyyy in yyyy_to_replace:
    nba_df['YYYY'] = nba_df['YYYY'].replace(yyyy,"Other")
    
# Check to make sure binning was successful
nba_df['YYYY'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
nba_df = pd.get_dummies(nba_df)

In [ ]:
# Split our preprocessed data into our features and target arrays
y = nba_df['']
X = nba_df.drop(columns='')

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
from tensorflow import keras
from tensorflow.keras import layers

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(layers.Dense(units = 100, activation = 'relu', input_dim = ))

# Second hidden layer
nn.add(layers.Dense(units = 50, activation = 'elu'))

# Output layer
nn.add(layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss ='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# Train the model
fit_modelnn = nn.fit(X_train_scaled, y_train, epochs = 100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")